In [1]:
import tensorflow as tf 
import numpy as np 
import cv2 
import glob 

In [2]:
file='red_lights.jpg'
model_name = "ssd_resnet50_v1_fpn_640x640_coco17_tpu-8"
url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + model_name + '.tar.gz'
    
# It will download the model and return the directory where it is downloaded.
model_dir = tf.keras.utils.get_file(fname=model_name, untar=True, origin=url)

model_dir = str(model_dir) + "/saved_model"
model = tf.saved_model.load(str(model_dir))

In [3]:
file='images/images_today1.jpeg'
traffic_light_count = 0
# read images.
img_bgr = cv2.imread(file)
img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
input_tensor = tf.convert_to_tensor(img_rgb) # Input needs to be a tensor
input_tensor = input_tensor[tf.newaxis, ...]

# this will detect any tranffic lights in given images.
output = model(input_tensor)

# print("num_detections:", output['num_detections'], int(output['num_detections']))

# Convert the tensors to a NumPy array
num_detections = int(output.pop('num_detections'))
output = {key: value[0, :num_detections].numpy()
        for key, value in output.items()}
output['num_detections'] = num_detections

# print('Detection classes:', output['detection_classes'])
# print('Detection Boxes:', output['detection_boxes'])

# The detected classes need to be integers.
output['detection_classes'] = output['detection_classes'].astype(np.int64)
output['boxes'] = [
{"y": int(box[0] * img_rgb.shape[0]), "x": int(box[1] * img_rgb.shape[1]), "y2": int(box[2] * img_rgb.shape[0]),
    "x2": int(box[3] * img_rgb.shape[1])} for box in output['detection_boxes']]


out = output


In [9]:
traffic_light_count = 0
for idx in range(len(out['boxes'])):
 
    # Extract the type of object that was detected  
    obj_class = out["detection_classes"][idx]
    score = int(out["detection_scores"][idx] * 100)
    # If the object that was detected is a traffic light
    if obj_class == 10:
         
      # Extract the coordinates of the bounding box
      box = out["boxes"][idx]
             
      # Extract (i.e. crop) the traffic light from the image     
      traffic_light = img_rgb[box["y"]:box["y2"], box["x"]:box["x2"]]
             
      # Convert the traffic light from RGB format into BGR format
      traffic_light = cv2.cvtColor(traffic_light, cv2.COLOR_RGB2BGR)
 
      if score > 20:
        # Store the cropped image in a folder named 'traffic_light_cropped'     
        cv2.imwrite("traffic_light_cropped_detected3/" + str(traffic_light_count) + ".jpg", traffic_light)
        print('written to file.')     
        # Increment the number of traffic lights by 1
        traffic_light_count = traffic_light_count + 1
 
# Display the total number of traffic lights identified
print("Number of Traffic lights identified:", traffic_light_count)

written to file.
written to file.
written to file.
written to file.
Number of Traffic lights identified: 4
